In [ ]:
import numpy as np
import soundfile as sf
from groq import Groq
import io
import speech_recognition as sr
import pyttsx3  # Using for free offline TTS

# ---------------------------------------------------------
# 🔑 YOUR API KEYS (replace these)
# ---------------------------------------------------------
GROQ_API_KEY = "xxxxxxxxxxxxxxxxxxxxxxxxxxxxx"  # <-- Make sure to paste your key here

# ---------------------------------------------------------
# ✨ CLIENTS
# ---------------------------------------------------------
# We only need to initialize the Groq client here.
# The TTS engine will be initialized inside the 'speak' function.
try:
    groq_client = Groq(api_key=GROQ_API_KEY)
except Exception as e:
    print(f"Error initializing Groq client: {e}")
    exit()

# ---------------------------------------------------------
# 🎧 AUDIO SETTINGS
# ---------------------------------------------------------
SAMPLE_RATE = 12000

# ---------------------------------------------------------
# 🤖 TEXT → SPEECH (Correct, Working Version)
# ---------------------------------------------------------
def speak(text_to_speak):
    """
    Converts text to speech using the free offline pyttsx3 engine.
    The engine is initialized and *fully shut down* inside this
    function to prevent conflicts with the microphone library.
    """
    print(f"\nAI: {text_to_speak}")
    print("AI: *Speaking...*")

    tts_engine = None  # Define it so 'finally' can see it
    try:
        # 1. Initialize a fresh engine EVERY time
        tts_engine = pyttsx3.init()

        # 2. Say the text
        tts_engine.say(text_to_speak)

        # 3. Block until speaking is finished
        tts_engine.runAndWait()

    except Exception as e:
        print(f"Error during TTS speech: {e}")

    finally:
        # 4. Forcefully shut down the engine
        # This is the crucial step to "unstick" the audio
        if tts_engine:
            tts_engine.stop()
            del tts_engine

# ---------------------------------------------------------
# 🎤 SPEECH → TEXT (Free Google Speech Recognition)
# ---------------------------------------------------------
def transcribe_audio_free():
    """
    Records audio from the microphone and transcribes it using
    the free Google Web Speech API.
    """
    r = sr.Recognizer()

    with sr.Microphone(sample_rate=SAMPLE_RATE) as source:
        print("\n🎤 Listening... Speak now!")
        r.adjust_for_ambient_noise(source, duration=1.5)

        try:
            audio_data = r.listen(source, timeout=18, phrase_time_limit=10)
            print("✅ Got it. Transcribing...")

            text = r.recognize_google(audio_data)
            print(f"You: {text}")
            return text

        except sr.WaitTimeoutError:
            print("Listen timeout. No speech detected.")
            return None
        except sr.UnknownValueError:
            print("Google Speech Recognition could not understand audio.")
            return None
        except sr.RequestError as e:
            print(f"Could not request results from Google; {e}")
            return None
        except Exception as e:
            print(f"Error during transcription: {e}")
            return None

# ---------------------------------------------------------
# 🧠 CHAT (Free Groq Llama 3 Model)
# ---------------------------------------------------------
def get_groq_response(prompt_text):
    """
    Gets a chat response from Groq's Llama 3 model.
    """
    print("🧠 AI is thinking (fast)...")
    try:
        response = groq_client.chat.completions.create(
            model="llama-3.1-8b-instant",
            messages=[
                {"role": "system", "content": "You are a polite dental receptionist named Dentora."},
                {"role": "user", "content": prompt_text}
            ],
            max_tokens=150,
        )
        return response.choices[0].message.content
    except Exception as e:
        print(f"Error getting Groq response: {e}")
        return "I'm sorry, I seem to be having trouble connecting."

# ---------------------------------------------------------
# 🔄 MAIN CONVERSATION LOOP
# ---------------------------------------------------------
def main_loop():
    # Initial greeting
    speak("Thank you for calling Appollo Hospital. My name is Dentora , how may I assist you?")

    try:
        while True:
            # 1. Listen (Free)
            user_text = transcribe_audio_free()

            if user_text:
                if "goodbye" in user_text.lower():
                    speak("Thank you for calling. Have a great day!")
                    break

                # 2. Think (Free)
                ai_response = get_groq_response(user_text)

                # 3. Speak (Free)
                if ai_response:
                    speak(ai_response)
            else:
                print("Didn't catch that, listening again.")

    except KeyboardInterrupt:
        print("\n👋 Goodbye!")

# ---------------------------------------------------------
# 🚀 START BOT
# ---------------------------------------------------------
if __name__ == "__main__":
    main_loop()